## WmdSimilarityで類似検索のサンプルを実装してみる
- 参考
    - https://markroxor.github.io/gensim/static/notebooks/WMD_tutorial.html

In [17]:
'''
    プロトタイピング用のパスと、Botライブラリーパスを取得／設定します
'''
import sys
import os

prototype_dir = os.path.join(os.getcwd(), '..')
prototype_dir = os.path.abspath(prototype_dir)

learning_dir = os.path.join(prototype_dir, '..')
learning_dir = os.path.abspath(learning_dir)
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

print('prototype_dir=%s\nlearning_dir=%s' % (prototype_dir, learning_dir))

prototype_dir=/Users/harada/source_code/donusagi-bot/learning/prototype
learning_dir=/Users/harada/source_code/donusagi-bot/learning


In [21]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('prototype/word2vec_WMD/model.vec', binary=False) 
print('model loaded')

model loaded


In [15]:
'''
文章群から類似文章を検索するサンプル
WmdSimilarityのインスタンスを生成する処理がそこそこ遅い
'''
import numpy as np
from IPython.core.display import display
from gensim.similarities import WmdSimilarity

corpus = np.array([
    ['こんにちは', '交通', '費', '申請'],
    ['海外', '日当' ],
    ['国内', '出張', '担当', '者', '教え', '下さい'],
    ['海外', '出張', '申請', 'ルート', '承認', '者', '変更']
])
display(corpus)

instance = WmdSimilarity(corpus, model, num_best=10)
sims = instance[['こんにちは', '交通']]
display(sims)

array([list(['こんにちは', '交通', '費', '申請']), list(['海外', '日当']),
       list(['国内', '出張', '担当', '者', '教え', '下さい']),
       list(['海外', '出張', '申請', 'ルート', '承認', '者', '変更'])], dtype=object)

[(0, 0.60625908656304373),
 (2, 0.44349872101479948),
 (3, 0.4384873174086511),
 (1, 0.43675135804911841)]

In [35]:
'''
豊通2017/10/12時点の本番データ326件に対してWMDで検索をかける
WmdSimilarityのインスタンスを生成するところが少し遅いが、
事前にオンメモリにしておけば十分使えそうな印象
'''
from app.core.tokenizer.mecab_tokenizer import MecabTokenizer
from app.shared.config import Config
import pandas as pd

Config().init('development')

data = pd.read_csv('prototype/word2vec_WMD/toyotsu.csv')
display(data.shape)

questions = data['質問']
tokenized_sentences = MecabTokenizer().tokenize(questions)
display(tokenized_sentences)

instance = WmdSimilarity(tokenized_sentences, model, num_best=10)
sims = instance['国内 出張 担当 者 教える']
display(sims)

for i, sim in sims:
    display(questions[i])

(329, 3)

[DEBUG 2017/10/12 PM 11:35:28           mecab_tokenizer:  9 -             __init__] dicdir: /usr/local/lib/mecab/dic/mecab-ipadic-neologd


['こんにちは',
 'ありがとう',
 'ありがとう',
 'さようなら',
 'こんばんは',
 'こんばんわ',
 'おはよう',
 'おはよう',
 'お疲れ様',
 'またね',
 'お世話 なる',
 'お世話になりました',
 '質問',
 'おやすみなさい',
 'はじめまして',
 'あけましておめでとう',
 'メリークリスマス',
 'サンキュー。',
 '山九',
 'ごきげんよう',
 'バイバイ。',
 'こんにちわ',
 '国内 出張 天候 影響 新幹線 遅延 する 一部 払い戻し する れる 場合 精算 方法',
 '国内 出張 ＪＲ 特急券 どう やる 精算 する 良い',
 '宿泊 伴う 国内 出張 日当 いくら なる',
 '出向 中 国内 出張 旅費 精算 する 良い',
 '急 辞令 異動 なる 社宅 手配 間に合う 月 中 移動 なる それ 実家 通勤 なる 通勤費 どう する 良い',
 '急 辞令 異動 なる 社宅 手配 間に合う 引越し 複数回 往復 する なる 場合 交通費 赴任 旅費',
 '国内 宿泊 費 規定 安い 申請 どう する 良い',
 '休日 出張 する 日当 出る',
 '国内 出張 間 休日 場合 働く ない 日当 出る',
 '国内 出張 コンサルタント 交通費 負担 する 場合 通常 勘定科目 良い',
 '出向 解除 なる 出張 １ ヶ月 延長 する 出向 先 勤務 する なる 場合 交通費 通勤費 どう 精算 する 良い',
 '赴任 旅費 精算 する 赴任 旅費 精算 書 領収書 提出 する 良い',
 '有料道路 領収書 紛失 する 場合 精算 どう する 良い',
 '国内 出張 １枚 領収書 ２つ 部署 それぞれ ６０ ％ ４０ ％ 精算 する 場合 どう 申請 する 良い',
 '国内 出張 留守 宅 宿泊 する 場合 宿泊 費 出る ない 留守 宅 何',
 '国内 出張 時 航空券 取り消し 料 非課税 勘定科目 何 なる',
 '国内 出張 時 移動 開始 労働 終了 １１時間 場合 日当 どう なる',
 '国内 出張 時 航空券 購入 勘定科目 何 なる',
 'コーポレート カード 持つ ない 金券ショップ 新幹線 回数券 購入 する 精算 申請 する 良い',
 

[(325, 1.0),
 (324, 0.81566456378628427),
 (150, 0.66710688873823365),
 (51, 0.64145893123344999),
 (41, 0.60543811554158433),
 (65, 0.5964856622103577),
 (62, 0.59646060307227744),
 (60, 0.59498362935754256),
 (64, 0.58512340785428441),
 (77, 0.5825393942370013)]

'国内出張担当者を教えて下さい。'

...

'国内出張について、承認者の変更の仕方は？'

'国内出張について、承認者の初期設定は？'

'国内出張について、国内出張旅費システムの利用対象者は？'

'国内出張について、費用負担先の部署承認者は？'

'国内出張について、研修の場合日当は付きますか？'

In [50]:
'''
「海外出張担当者を教えて下さい。」を選択したかったが、 
このケースでは「国内出張担当者を教えて下さい。」が選択されてしまった。
'''
sims = instance['外国 出張 担当 者 教える']
display(sims)
for i, sim in sims:
    display(questions[i])

[(325, 0.90564000303744607),
 (324, 0.89142238032727283),
 (150, 0.66699191035898264),
 (41, 0.60547902369000006),
 (177, 0.60231103033118027),
 (51, 0.60127312603151639),
 (261, 0.58140110460630889),
 (265, 0.5771907171543792),
 (29, 0.57697891450392569),
 (65, 0.56469764595112693)]

'国内出張担当者を教えて下さい。'

'海外出張担当者を教えて下さい。'

'出張旅費の担当者が知りたい'

'海外出張から深夜便で帰国した際、国内出張の日当も付きますか？'

...

'国内出張の負担部署の内訳を教えてほしい'

'国内出向者が海外出張する場合の支度金について'

'国内出向者が海外出張する際の承認先・申請先は？'

'休日に出張した際、日当は出ますか？'

'国内出張について、承認者の変更の仕方は？'

In [51]:
'''
「じゃあね」という単語はデータ側には存在しないが、
意味的に正しく対応される「またね」が選択された。
'''
sims = instance['じゃあね']
display(sims)
for i, sim in sims:
    display(questions[i])

[(9, 0.54861560265697262),
 (1, 0.52158457642762979),
 (2, 0.52158457642762979),
 (14, 0.5106654593324893),
 (5, 0.49904981526220082),
 (15, 0.49800115024984393),
 (19, 0.49383477972511264),
 (4, 0.49377428617361102),
 (21, 0.49220286086943044),
 (7, 0.48794992243413038)]

'またね'

'ありがとう'

'ありがとうございました'

'はじめまして'

...

'あけましておめでとう'

'ごきげんよう'

'こんばんは'

'こんにちわ'

'おはよう'

In [55]:
'''
「休日に出張した際、日当は出ますか？」と選択されることを期待し、その通りの振る舞いになった。
「休日」と「休み」が類似判断されているような挙動。
'''
sims = instance['休み 手当 出張']
display(sims)
for i, sim in sims:
    display(questions[i])

[(29, 0.60570656163447445),
 (150, 0.55289675606385302),
 (325, 0.54108746012782649),
 (324, 0.54042339864500144),
 (294, 0.53781803373439252),
 (85, 0.53759282910881245),
 (30, 0.53607391326530962),
 (41, 0.5299114096042753),
 (86, 0.52715477649235609),
 (262, 0.52713400731294147)]

'休日に出張した際、日当は出ますか？'

'出張旅費の担当者が知りたい'

'国内出張担当者を教えて下さい。'

'海外出張担当者を教えて下さい。'

...

'国内出張について、1週間出張で同じ地にいました。\nこの場合は長期出張になりますか？'

'国内出張の間に休日がある場合、働いていなくても日当は出ますか？'

'海外出張から深夜便で帰国した際、国内出張の日当も付きますか？'

'国内出張について、1ヶ月以上同じ地での長期出張の場合、宿泊費と日当は通常の出張と異なりますか？'

'海外出張時の仮出金はいくらまで？'

In [57]:
'''
「休日に出張した際、日当は出ますか？'」が選択されることを期待したが、異なる結果を返した。
'''
sims = instance['休日 日当']
display(sims)
for i, sim in sims:
    display(questions[i])

[(327, 0.6150274374733592),
 (328, 0.60759980918081224),
 (29, 0.57169717249796526),
 (30, 0.51358814285173382),
 (240, 0.50429020404896052),
 (77, 0.497418534566342),
 (310, 0.4865702685510796),
 (82, 0.48510542560559122),
 (24, 0.48501522244627926),
 (73, 0.48483021056010017)]

'国内の日当について'

'海外の日当について'

'休日に出張した際、日当は出ますか？'

'国内出張の間に休日がある場合、働いていなくても日当は出ますか？'

...

'国内出張について、研修の場合日当は付きますか？'

'長期海外出張で、日当のみ途中精算したいときの「帰国日」は？「レート」は？'

'国内出張について、日帰り日当の所要時間は自宅を出た時刻からの計算ではダメですか？'

'宿泊を伴う国内出張の日当はいくらになりますか？'

'国内出張について、決算月の場合、いつまでのものが当月計上になりますか？'

In [58]:
'''
「３ヶ月の海外出張の場合の出張費精算の仕方は、どうやりますか？'」が選択されることを期待しその通りになった。
同一の単語ベクトルが多いので当然の結果と言える。
'''
sims = instance['海外出張 精算 仕方']
display(sims)
for i, sim in sims:
    display(questions[i])

[(170, 0.61364951984536964),
 (167, 0.59077236097387198),
 (313, 0.58636593218423549),
 (289, 0.58288009934805363),
 (262, 0.5812172297785464),
 (155, 0.57923282146171218),
 (281, 0.57861759596937523),
 (180, 0.57836045685249926),
 (324, 0.57305001879400019),
 (294, 0.57108765804023942)]

'３ヶ月の海外出張の場合の出張費精算の仕方は、どうやりますか？'

'海外出張中に現地で円貨で支払ったが精算方法は？'

'海外出張精算について、「費用計算書」の東京の提出場所はどこですか？'

'海外出張時の勤務表の入力は？'

...

'海外出張の精算届の実績を、データとして抽出できますか？'

'海外出張精算の四半期決算月の処理について、教えて下さい。'

'１ヶ月以上の長期海外出張の場合の精算処理方法は？'

'海外出張担当者を教えて下さい。'

'海外出張時のビザ手続きは？'

In [59]:
'''
「３ヶ月の海外出張の場合の出張費精算の仕方は、どうやりますか？'」が選択されることを期待しが、異なる結果になった。
「方法」と「仕方」の対応付けを期待したが、類似よりも一致の方が優先された。
'''
sims = instance['海外出張 精算 方法']
display(sims)
for i, sim in sims:
    display(questions[i])

[(167, 0.61630148320798805),
 (180, 0.60324616150615862),
 (170, 0.59212022579578605),
 (313, 0.58405506878469138),
 (262, 0.57998027611723002),
 (155, 0.57690779130033565),
 (281, 0.57627609436505933),
 (289, 0.57399245524778686),
 (324, 0.5721341843130624),
 (294, 0.56967763645205516)]

'海外出張中に現地で円貨で支払ったが精算方法は？'

'１ヶ月以上の長期海外出張の場合の精算処理方法は？'

'３ヶ月の海外出張の場合の出張費精算の仕方は、どうやりますか？'

'海外出張精算について、「費用計算書」の東京の提出場所はどこですか？'

...

'海外出張の精算届の実績を、データとして抽出できますか？'

'海外出張精算の四半期決算月の処理について、教えて下さい。'

'海外出張時の勤務表の入力は？'

'海外出張担当者を教えて下さい。'

'海外出張時のビザ手続きは？'

In [60]:
'''
「'海外出張システムのマニュアルは？'」が選択されることを期待しその通りになった。
同一の単語ベクトルが多いので当然の結果と言える。
'''
sims = instance['海外出張 マニュアル']
display(sims)
for i, sim in sims:
    display(questions[i])

[(241, 0.74727506750840478),
 (286, 0.59912209613849388),
 (294, 0.57756049498942319),
 (284, 0.57528414859921395),
 (262, 0.57092730233676936),
 (289, 0.56947403949331632),
 (324, 0.56933961248330367),
 (320, 0.55640258860751612),
 (255, 0.5560028956358003),
 (272, 0.54970883095525125)]

'海外出張システムのマニュアルは？'

'海外出張時のハンドキャリーについて'

'海外出張時のビザ手続きは？'

'年末年始の海外出張システムは？'

...

'海外出張時の勤務表の入力は？'

'海外出張担当者を教えて下さい。'

'海外出張申請を提出するのを忘れていました。'

'海外出張中に病院に行った場合'

'国内出張と同様に、海外出張時の宿泊費の上限はありますか？'

In [61]:
'''
「'海外出張システムのマニュアルは？'」が選択されることを期待しその通りになった。
「マニュアル」と「説明書」の対応付けを期待したが、全く類似性が含まれていない結果になった。
'''
sims = instance['海外出張 説明書']
display(sims)
for i, sim in sims:
    display(questions[i])

[(262, 0.59035231162394253),
 (289, 0.57362873427157013),
 (324, 0.57224965684826756),
 (294, 0.56992072585390552),
 (272, 0.56600997760463911),
 (320, 0.56502835496656401),
 (255, 0.55782237156911274),
 (286, 0.54975564246019926),
 (253, 0.548646816064568),
 (173, 0.54726066593080336)]

'海外出張時の仮出金はいくらまで？'

'海外出張時の勤務表の入力は？'

'海外出張担当者を教えて下さい。'

'海外出張時のビザ手続きは？'

...

'海外出張申請を提出するのを忘れていました。'

'海外出張中に病院に行った場合'

'海外出張時のハンドキャリーについて'

'海外出張時の会社付保の保険内容は？'

'海外出張時のレンタカーの許可申請書は、ありますか？'

## 結論
- 一部word2vec + WMDにすることにより、単語の意味的類似性を含んだ検索がなされているように感じた
- 大きく精度が落ちるような挙動は発生していないように思えるが、同様に大きく改善しているようにも思えなかった
- 害はあまりなさそうなのでプロダクションコードに実装を進めたい